In [1]:
import faiss
import torch
import wget
import os
import numpy as np
import pandas as pd
from IPython.display import display
from tqdm import tqdm

In [2]:
def predict_partition(test_kb_embs, test_kb_cids, test_query_embs, k, queries_len):
    # Initialize knowledge base and query data loaders
    #results_top_1 = []
    results = []
    result_ids = []
    result_scores = []
    #test_kb_dataloader = model.setup_dataloader(test_kb, is_index_data=True)
    #test_query_dataloader = model.setup_dataloader(test_queries, is_index_data=True)
    
    # Get knowledge base and query embeddings
    #test_kb_embs, test_kb_cids = get_embeddings(model, test_kb_dataloader)
    #test_query_embs, test_query_cids = get_embeddings(model, test_query_dataloader)
    #test_query_embs = get_query_embedding(queries, model).cpu().detach().numpy()

    # Calculate the cosine distance between each query and knowledge base concept
    score_matrix = np.matmul(np.array(test_query_embs), np.array(test_kb_embs).T)
    #accs = {k : 0 for k in ks}
    
    # Compare the knowledge base IDs of the knowledge base entities with 
    # the smallest cosine distance from the query 
    for query_idx in tqdm(range(queries_len)):
        query_emb = test_query_embs[query_idx]
        #query_cid = test_query_cids[query_idx]
        query_scores = score_matrix[query_idx]

        #for k in ks:
        topk_idxs = np.argpartition(query_scores, -k)[-k:]
        topk_cids = [test_kb_cids[idx].item() for idx in topk_idxs]
        topk_scores = [query_scores[idx] for idx in topk_idxs]
            
        #    if k == 1:
        results.append(topk_cids)
        result_ids.append(topk_idxs)
        result_scores.append(topk_scores)
            # If the correct query ID is amoung the top k closest kb IDs
            # the model correctly linked the entity
         #   match = int(query_cid in topk_cids)
         #   accs[k] += match

    #for k in ks:
    #    accs[k] /= len(test_query_cids)
                
    #return accs, results_top_1
    return results, result_ids, result_scores

In [3]:
path = "/mnt/d/school/data/umls-2023AA-metathesaurus-full/umls_sapbert/files"
k = 1
#load query embeddings - 2 files

gold_test_embs = np.load(f'{path}/umls_gold_test_627.npy')
#gold_test_embs = np.load(f'{path}/umls_bg_gold_test_5.npy')
#test_embs = np.load(f'{path}/umls_bg_pred_test_llama_6.npy')
#test_embs = np.load(f'{path}/umls_test_665.npy') 
test_embs = np.load(f'{path}/umls_mm_pred_test_llama_429.npy') 
#filename = 'gold_test'
filename_gold = 'mm_gold_test'
filename_pred = 'mm_test_llama'

#filename_gold = 'bg_gold_test'
#filename_pred = 'bg_test'
queries_len = 16000
gold_test_embs_len = len(gold_test_embs)
test_embs_len = len(test_embs)

microbatches = test_embs_len // queries_len + 1

In [4]:
microbatches

2

In [5]:
def store_results(results, result_ids, result_scores, batch_idx, microbatch_idx, filename, model_folder=''): #/xmlr
    with open(f'files/{model_folder}umls_{filename}llama_cids_{batch_idx}_{microbatch_idx}.npy', 'wb') as f:
        np.save(f, np.array(results))                
    with open(f'files/{model_folder}umls_{filename}llama_idx_{batch_idx}_{microbatch_idx}.npy', 'wb') as f:
        np.save(f, result_ids)
    with open(f'files/{model_folder}umls_{filename}llama_scores_{batch_idx}_{microbatch_idx}.npy', 'wb') as f:
        np.save(f, result_scores)

In [ ]:
for file_idx in range(1, 133): #xlmr - 134
    # load partition KB
    # calculate cosine similiraty
    test_kb_cids = np.load(f'{path}/umls_kb_cids_{file_idx}000.npy') #.0 cuis
    test_kb_embs = np.load(f'{path}/umls_kb_{file_idx}000.npy')
    
    for microbatch_idx in range(0, microbatches):
        start_idx = microbatch_idx * queries_len
        end_idx = start_idx + queries_len if start_idx + queries_len <= gold_test_embs_len else gold_test_embs_len
        
        results, result_ids, result_scores = predict_partition(test_kb_embs, test_kb_cids, gold_test_embs[start_idx:end_idx], k, end_idx-start_idx)
        store_results(results, result_ids, result_scores, file_idx, microbatch_idx, filename_gold)
        results, result_ids, result_scores = [], [], []
        
        start_idx = microbatch_idx * queries_len
        end_idx = start_idx + queries_len if start_idx + queries_len <= test_embs_len else test_embs_len
        
        results, result_ids, result_scores = predict_partition(test_kb_embs, test_kb_cids, test_embs[start_idx:end_idx], k, end_idx-start_idx)
        store_results(results, result_ids, result_scores, file_idx, microbatch_idx, filename_pred)
        results, result_ids, result_scores = [], [], []
        print(f"processed {file_idx}:{microbatch_idx}")
    
    test_kb_cids, test_kb_embs = [], []   

In [7]:
test_embs_len

27446

In [8]:
gold_test_embs_len

40097

In [12]:
filename = filename_pred #gold #'test'
num_samples = test_embs_len #gold_test_embs_len #42553 #40097
num_files = 133

total_results = np.empty((num_samples, num_files), dtype=object)
total_result_ids = np.zeros((num_samples, num_files))
total_result_scores = np.zeros((num_samples, num_files))

In [13]:
model_folder = '' #/xlmr
for file_idx in range(1, 133):
    # load partition KB
    column_results = []
    column_result_ids = []
    column_result_scores = []
    for microbatch_idx in range(0, microbatches):
        results = np.load(f'files/{model_folder}umls_{filename}llama_cids_{file_idx}_{microbatch_idx}.npy')
        result_ids = np.load(f'files/{model_folder}umls_{filename}llama_idx_{file_idx}_{microbatch_idx}.npy')
        result_scores = np.load(f'files/{model_folder}umls_{filename}llama_scores_{file_idx}_{microbatch_idx}.npy')
        
        column_results.extend(results)
        column_result_ids.extend(result_ids)
        column_result_scores.extend(result_scores)
        
    total_results[:,file_idx-1] = np.array(column_results).reshape((num_samples,))
    total_result_ids[:,file_idx-1] = np.array(column_result_ids).reshape((num_samples,))
    total_result_scores[:,file_idx-1] = np.array(column_result_scores).reshape((num_samples,))
    
    #column_results, column_result_ids, column_result_scores = [], [], []
    
    #break

In [14]:
total_results[0:10]

array([[45660, 113976, 142999, ..., 8366837, 8397265, None],
       [45660, 113976, 142999, ..., 8366837, 8397265, None],
       [45660, 113976, 142999, ..., 8366837, 8397265, None],
       ...,
       [39225, 70722, 153914, ..., 8323556, 8395603, None],
       [32120, 72448, 134350, ..., 8325945, 8388777, None],
       [32120, 72448, 134350, ..., 8325945, 8388777, None]], dtype=object)

In [15]:
total_result_scores[0:10]

array([[0.60376561, 0.59530568, 0.5888555 , ..., 0.57505178, 0.50889796,
        0.        ],
       [0.60376561, 0.59530568, 0.5888555 , ..., 0.57505178, 0.50889796,
        0.        ],
       [0.60376561, 0.59530568, 0.5888555 , ..., 0.57505178, 0.50889796,
        0.        ],
       ...,
       [0.7702142 , 0.84051204, 0.6441986 , ..., 0.59693217, 0.58231568,
        0.        ],
       [0.61764002, 0.59166759, 0.62796354, ..., 0.57642281, 0.7087428 ,
        0.        ],
       [0.61764002, 0.59166759, 0.62796354, ..., 0.57642281, 0.7087428 ,
        0.        ]])

In [16]:
total_result_scores.shape

(27446, 133)

In [17]:
topk_idxs = np.argmax(total_result_scores, axis=1)

In [18]:
#topk_idxs = np.argpartition(total_result_scores.T, -k)[-k:]

In [19]:
len(topk_idxs)

27446

In [20]:
np.max(topk_idxs)

131

In [21]:
topk_cids = [total_results[idx][top_idx] for idx, top_idx in zip(range(0,num_samples),topk_idxs)]
topk_scores = [total_result_scores[idx][top_idx] for idx, top_idx in zip(range(0,num_samples),topk_idxs)]

In [22]:
topk_cids[0:10]

[524641,
 524641,
 524641,
 2169616,
 2169616,
 712731,
 712731,
 712731,
 580440,
 580440]

In [23]:
topk_scores[0:10]

[1.0,
 1.0,
 1.0,
 0.7669359445571899,
 0.7669359445571899,
 1.0,
 1.0,
 1.0,
 0.949481189250946,
 0.949481189250946]

In [24]:
len(topk_scores)

27446

In [25]:
df_kb = pd.read_csv('umls_en_kb.tsv', sep='\t', header=None)
df_kb.head()

,0,1
0,C0000005,(131)I-Macroaggregated Albumin
1,C0000005,(131)I-MAA
2,C0000039,"1,2-dipalmitoylphosphatidylcholine"
3,C0000039,"1,2-dipalmitoylphosphatidylcholine"
4,C0000039,"1,2-Dipalmitoylphosphatidylcholine"


In [26]:
topk_cids_cui = [df_kb.iloc[idx][0] for idx in topk_cids]

In [27]:
topk_cids_cui[0:10]

['C0068946',
 'C0068946',
 'C0068946',
 'C0608491',
 'C0608491',
 'C0162638',
 'C0162638',
 'C0162638',
 'C0085262',
 'C0085262']

In [28]:
df_medmentions_test = pd.read_csv('st21pv_test_0928v1llama13b_entities_nofuzzy.tsv', sep='\t', header=0) 
#df_medmentions_test = pd.read_csv('bg_gold_standard_entities-predicted_lev_norm.tsv', sep='\t', header=0) 
#bg_gold_standard_entities-predicted_lev_norm bg_test_0813v113b_entities_nofuzzy
#gs_st21pv_test_entities
df_medmentions_test.head()

,document_id,sentence_id,doc_start_index,doc_end_index,sentence_entity_start,sentence_entity_end,text_segment
0,25847295,0,0,513,66,77,Nonylphenol
1,25847295,0,0,513,94,105,nonylphenol
2,25847295,0,0,513,370,381,nonylphenol
3,25847295,0,0,513,12,24,diethoxylate
4,25847295,0,0,513,382,394,diethoxylate


In [31]:
df_query_file = pd.read_csv('mm_llama_umls_query.tsv', sep='\t', header=None) 
df_query_file.shape
#27884439	1	654	1061	319	321	NA - 14331

(27446, 2)

In [32]:
df_query_file.head()

,0,1
0,0,Nonylphenol
1,1,nonylphenol
2,2,nonylphenol
3,3,diethoxylate
4,4,diethoxylate


In [29]:
df_medmentions_test.shape

(27447, 7)

In [90]:
#df_medmentions_test = df_medmentions_test[~df_medmentions_test['span'].isnull()]

In [37]:
df_medmentions_test['code'] = topk_cids_cui[0:14330] + [''] + topk_cids_cui[14330:]
df_medmentions_test['code'] = df_medmentions_test['code'].apply(lambda x: 'UMLS:'+x)

In [38]:
df_medmentions_test.head()

,document_id,sentence_id,doc_start_index,doc_end_index,sentence_entity_start,sentence_entity_end,text_segment,code
0,25847295,0,0,513,66,77,Nonylphenol,UMLS:C0068946
1,25847295,0,0,513,94,105,nonylphenol,UMLS:C0068946
2,25847295,0,0,513,370,381,nonylphenol,UMLS:C0068946
3,25847295,0,0,513,12,24,diethoxylate,UMLS:C0608491
4,25847295,0,0,513,382,394,diethoxylate,UMLS:C0608491


In [40]:
df_medmentions_test.iloc[14331]

document_id                   27884439
sentence_id                          1
doc_start_index                    654
doc_end_index                     1061
sentence_entity_start              352
sentence_entity_end                359
text_segment                   Dorpers
code                     UMLS:C1489583
Name: 14331, dtype: object

In [41]:
#df_medmentions_test.to_csv('st21pv_test_entities_pred_sap.tsv', sep='\t', index=False)
df_medmentions_test.to_csv('st21pv_test_0928v1llama13b_entities_nofuzzy_linked_sap.tsv', sep='\t', index=False)
#df_medmentions_test.to_csv('bg_gold_standard_entities-predicted_sap_xlmr_norm.tsv', sep='\t', index=False)

In [ ]:
#bg_gold_standard_entities-manual_translated
#bg_test_0813v113b_entities_nofuzzy